In [155]:
import numpy as np
import gzip
import struct
import matplotlib.pyplot as plt

In [156]:
def load_data(data_path, labels_path):
    with gzip.open(data_path, 'rb') as gz:
        print(struct.unpack('>bbbbIII', gz.read(16)))
        data = np.frombuffer(gz.read(), np.uint8).reshape(-1, 28, 28)
    with gzip.open(labels_path, 'rb') as gz:
        print(struct.unpack('>bbbbI', gz.read(8)))
        labels = np.frombuffer(gz.read(), np.uint8)
    return data, labels

In [157]:
# plt.subplot(1, 2, 1)
# plt.title(str(train_labels[0]))
# plt.imshow(train_images[0], 'gray')
#
# plt.subplot(1, 2, 2)
# plt.title(str(test_labels[0]))
# plt.imshow(test_images[0], 'gray')

In [166]:
def one_hot_encoder(labels):
    uniques = np.unique(labels)  # get unique elements
    uniques.sort()  #sort them
    uniques = uniques[::-1]  #reverse array order
    one_hot_translation = {}  #create an empty dictionary with unique items as keys, and one hot translation as value
    for item in uniques:
        one_hot_translation[item] = np.zeros(uniques.shape, dtype=int)
        one_hot_translation[item][np.where(uniques == item)[0]] = 1
    return one_hot_translation
    #
    # table = np.zeros((labels.shape[0], 10))
    # for i in range(labels.shape[0]):
    #     table[i][int(labels[i][0])] = 1
    # return table

In [167]:
class NeuralNetwork:
    def __init__(self, data, targets, batch=64, lr=1e-3, epochs=50):
        self.data = data
        self.target = targets
        self.batch = batch
        self.epochs = epochs
        self.lr = lr

        self.x = self.data[:self.batch]  # batch input
        self.y = self.target # batch target value
        self.loss = []
        self.acc = []

        self.init_weights()

    def init_weights(self):
        self.W1 = np.random.randn(self.data.shape[1], 256)
        self.W2 = np.random.randn(self.W1.shape[1], 128)
        self.W3 = np.random.randn(self.W2.shape[1], self.y.shape[0])

        self.b1 = np.random.randn(self.W1.shape[1], )
        self.b2 = np.random.randn(self.W2.shape[1], )
        self.b3 = np.random.randn(self.W3.shape[1], )

    def ReLU(self, x):
        return np.maximum(0, x)

    def dReLU(self, x):
        return 1 * (x > 0)

    def softmax(self, z):
        z = z - np.max(z, axis=1).reshape(z.shape[0], 1)
        return np.exp(z) / np.sum(np.exp(z), axis=1).reshape(z.shape[0], 1)

    def shuffle(self):
        idx = [i for i in range(self.data.shape[0])]
        np.random.shuffle(idx)
        self.data = self.data[idx]
        self.target = self.target[idx]

    def feedforward(self):
        assert self.x.shape[1] == self.W1.shape[0]
        self.z1 = self.x.dot(self.W1) + self.b1
        self.a1 = self.ReLU(self.z1)

        assert self.a1.shape[1] == self.W2.shape[0]
        self.z2 = self.a1.dot(self.W2) + self.b2
        self.a2 = self.ReLU(self.z2)

        assert self.a2.shape[1] == self.W3.shape[0]
        self.z3 = self.a2.dot(self.W3) + self.b3
        self.a3 = self.softmax(self.z3)
        self.error = self.a3 - self.y

    def backprop(self):
        dcost = (1 / self.batch) * self.error

        DW3 = np.dot(dcost.T, self.a2).T
        DW2 = np.dot((np.dot((dcost), self.W3.T) * self.dReLU(self.z2)).T, self.a1).T
        DW1 = np.dot((np.dot(np.dot((dcost), self.W3.T) * self.dReLU(self.z2), self.W2.T) * self.dReLU(self.z1)).T,
                     self.x).T

        db3 = np.sum(dcost, axis=0)
        db2 = np.sum(np.dot((dcost), self.W3.T) * self.dReLU(self.z2), axis=0)
        db1 = np.sum((np.dot(np.dot((dcost), self.W3.T) * self.dReLU(self.z2), self.W2.T) * self.dReLU(self.z1)),
                     axis=0)

        assert DW3.shape == self.W3.shape
        assert DW2.shape == self.W2.shape
        assert DW1.shape == self.W1.shape

        assert db3.shape == self.b3.shape
        assert db2.shape == self.b2.shape
        assert db1.shape == self.b1.shape

        self.W3 = self.W3 - self.lr * DW3
        self.W2 = self.W2 - self.lr * DW2
        self.W1 = self.W1 - self.lr * DW1

        self.b3 = self.b3 - self.lr * db3
        self.b2 = self.b2 - self.lr * db2
        self.b1 = self.b1 - self.lr * db1

    def train(self):
        for epoch in range(self.epochs):
            l = 0
            acc = 0
            self.shuffle()

            for batch in range(self.data.shape[0] // self.batch - 1):
                start = batch * self.batch
                end = (batch + 1) * self.batch
                self.x = self.data[start:end]
                self.y = self.target[start:end]
                self.feedforward()
                self.backprop()
                l += np.mean(self.error ** 2)
                acc += np.count_nonzero(np.argmax(self.a3, axis=1) == np.argmax(self.y, axis=1)) / self.batch

            self.loss.append(l / (self.data.shape[0] // self.batch))
            self.acc.append(acc * 100 / (self.data.shape[0] // self.batch))

    def plot(self):
        plt.figure(dpi=125)
        plt.plot(self.loss)
        plt.xlabel("Epochs")
        plt.ylabel("Loss")

    def acc_plot(self):
        plt.figure(dpi=125)
        plt.plot(self.acc)
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")

    def test(self, xtest, ytest):
        self.x = xtest
        self.y = ytest
        self.feedforward()
        acc = np.count_nonzero(np.argmax(self.a3, axis=1) == np.argmax(self.y, axis=1)) / self.x.shape[0]
        print("Accuracy:", 100 * acc, "%")

train_images, train_labels = load_data('./files/train-images-idx3-ubyte.gz', './files/train-labels-idx1-ubyte.gz')
test_images, test_labels = load_data('./files/t10k-images-idx3-ubyte.gz', './files/t10k-labels-idx1-ubyte.gz')

print("im1:", train_labels.shape)
train_targets = one_hot_encoder(train_labels)
test_targets = one_hot_encoder(test_labels)

NN = NeuralNetwork(train_images, train_targets)
NN.train()
NN.plot()
NN.test(test_images, test_targets)

(0, 0, 8, 3, 60000, 28, 28)
(0, 0, 8, 1, 60000)
(0, 0, 8, 3, 10000, 28, 28)
(0, 0, 8, 1, 10000)
im1: (60000,)


AttributeError: 'dict' object has no attribute 'shape'